# Test Plugin Integration

> Test the Gemini plugin with the transcription plugin system

In [1]:
import os
import logging
import json
import numpy as np
from pathlib import Path

# Import PluginManager from the generic plugin system
from cjm_plugin_system.core.manager import PluginManager
from cjm_transcription_plugin_system.plugin_interface import TranscriptionPlugin
from cjm_transcription_plugin_system.core import AudioData
from cjm_transcription_plugin_gemini.plugin import GeminiPlugin

## Test Direct Plugin Usage

In [2]:
# Create plugin directly
plugin = GeminiPlugin()

# Check basic properties
print(f"Plugin: {plugin.name} v{plugin.version}")
print(f"Available: {plugin.is_available()}")
print(f"Supported formats: {', '.join(plugin.supported_formats)}")

Plugin: gemini v1.0.0
Available: True
Supported formats: wav, mp3, aiff, aac, ogg, flac


In [3]:
# Get configuration class and defaults
from dataclasses import fields
from cjm_plugin_system.utils.validation import SCHEMA_DESC

config_class = plugin.config_class
defaults = plugin.get_config_defaults()

print("Configuration options:")
print(f"- Config class: {config_class.__name__}")
print(f"- Total properties: {len(fields(config_class))}")
print("\nKey configuration properties:")
for prop in ['model', 'temperature', 'downsample_audio', 'prompt']:
    f = next((f for f in fields(config_class) if f.name == prop), None)
    if f:
        print(f"  {prop}:")
        print(f"    Type: {f.type}")
        print(f"    Default: {defaults.get(prop)}")
        print(f"    Description: {f.metadata.get(SCHEMA_DESC, 'N/A')}")

Configuration options:
- Config class: GeminiPluginConfig
- Total properties: 17

Key configuration properties:
  model:
    Type: <class 'str'>
    Default: gemini-2.5-flash
    Description: Gemini model to use for transcription
  temperature:
    Type: <class 'float'>
    Default: 0.0
    Description: Sampling temperature
  downsample_audio:
    Type: <class 'bool'>
    Default: False
    Description: Downsample audio before uploading (requires ffmpeg)
  prompt:
    Type: <class 'str'>
    Default: Generate a transcription of the audio, only extract speech and ignore background audio.
    Description: Prompt for transcription


## Test with Plugin Manager

In [4]:
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

# Create plugin manager with TranscriptionPlugin interface
manager = PluginManager(plugin_interface=TranscriptionPlugin)

In [5]:
# Load plugin from module (for development)
import sys
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Create a temporary module file
temp_plugin_file = Path("temp_gemini_plugin.py")
with open(temp_plugin_file, "w") as f:
    f.write("from cjm_transcription_plugin_gemini.plugin import GeminiPlugin\n")

# Load the plugin
# Note: API key must be set in environment or config
config = {"model": "gemini-2.5-flash"}
if os.environ.get("GEMINI_API_KEY"):
    success = manager.load_plugin_from_module(str(temp_plugin_file), config=config)
    print(f"Plugin loaded: {success}")
else:
    print("Set GEMINI_API_KEY environment variable to load the plugin")
    success = False

# Clean up temp file
temp_plugin_file.unlink()

httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Loaded plugin from module: gemini


Plugin loaded: True


In [6]:
if success:
    # List loaded plugins
    print("Loaded plugins:")
    for meta in manager.list_plugins():
        print(f"  - {meta.name} v{meta.version} (enabled: {meta.enabled})")
    
    # Get available models
    gemini_plugin = manager.get_plugin("gemini")
    if hasattr(gemini_plugin, 'get_available_models'):
        models = gemini_plugin.get_available_models()
        print(f"\nAvailable models ({len(models)} total):")
        for model in models[:10]:  # Show first 10
            print(f"  - {model}")

httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"


Loaded plugins:
  - gemini v1.0.0 (enabled: True)


httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models



Available models (26 total):
  - nano-banana-pro-preview
  - gemma-3n-e4b-it
  - gemma-3n-e2b-it
  - gemma-3-4b-it
  - gemma-3-27b-it
  - gemma-3-1b-it
  - gemma-3-12b-it
  - gemini-robotics-er-1.5-preview
  - gemini-pro-latest
  - gemini-flash-lite-latest


In [7]:
if success:
    # Get current configuration (returns a dataclass)
    current_config = manager.get_plugin_config("gemini")
    print("Current Gemini configuration:")
    config_subset = {
        "model": current_config.model,
        "temperature": current_config.temperature,
        "top_p": current_config.top_p,
        "downsample_audio": current_config.downsample_audio,
        "safety_settings": current_config.safety_settings
    }
    print(json.dumps(config_subset, indent=2))

Current Gemini configuration:
{
  "model": "gemini-2.5-flash",
  "temperature": 0.0,
  "top_p": 0.95,
  "downsample_audio": false,
  "safety_settings": "OFF"
}


## Test Configuration Management

In [8]:
if success:
    # Test configuration validation using dataclass-based validation
    from cjm_plugin_system.utils.validation import dict_to_config, validate_config, SCHEMA_ENUM
    from cjm_transcription_plugin_gemini.plugin import GeminiPluginConfig
    
    test_configs = [
        ({"model": "gemini-2.0-flash"}, "Valid: switching to 2.0 flash"),
        ({"model": "invalid_model"}, "Invalid: bad model name"),
        ({"temperature": 0.7}, "Valid: adjusting temperature"),
        ({"temperature": 3.0}, "Invalid: temperature out of range"),
        ({"downsample_audio": True, "downsample_rate": 16000}, "Valid: enable downsampling"),
    ]
    
    for config_update, description in test_configs:
        print(f"{description}")
        print(f"  Config: {config_update}")
        try:
            # Get current config from plugin and merge with update
            current = gemini_plugin.get_current_config()
            merged = {**current.__dict__, **config_update} if current else config_update
            
            # Try to create a validated config
            test_config = dict_to_config(GeminiPluginConfig, merged, validate=True)
            print(f"  Valid: True")
        except ValueError as e:
            print(f"  Valid: False")
            print(f"  Error: {str(e)[:80]}...")
        print()

Valid: switching to 2.0 flash
  Config: {'model': 'gemini-2.0-flash'}
  Valid: True

Invalid: bad model name
  Config: {'model': 'invalid_model'}
  Valid: False
  Error: model: 'invalid_model' is not one of ['gemini-2.5-flash', 'gemini-2.5-flash-prev...

Valid: adjusting temperature
  Config: {'temperature': 0.7}
  Valid: True

Invalid: temperature out of range
  Config: {'temperature': 3.0}
  Valid: False
  Error: temperature: 3.0 is greater than maximum 2.0...

Valid: enable downsampling
  Config: {'downsample_audio': True, 'downsample_rate': 16000}
  Valid: True



In [9]:
if success:
    # Update configuration
    new_config = {
        "temperature": 0.3,
        "prompt": "Transcribe this audio accurately, including any technical terms.",
        "downsample_audio": True,
        "downsample_rate": 16000
    }
    
    update_success = manager.update_plugin_config("gemini", new_config, merge=True)
    print(f"Configuration updated: {update_success}")
    
    if update_success:
        updated_config = manager.get_plugin_config("gemini")
        print("\nUpdated configuration:")
        for key in new_config:
            # Access via attribute since it's a dataclass
            value = getattr(updated_config, key) if hasattr(updated_config, key) else updated_config.get(key)
            print(f"  {key}: {value}")

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Cleanup completed
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: gemini


Configuration updated: True

Updated configuration:
  temperature: 0.3
  prompt: Transcribe this audio accurately, including any technical terms.
  downsample_audio: True
  downsample_rate: 16000


## Test Transcription

In [10]:
from nbdev.config import get_config
from pathlib import Path

config = get_config()
project_dir = config.config_path
test_dir = project_dir/"./test_files/"
audio_path = test_dir/"short_test_audio.mp3"
# audio_path = test_dir/"constitution_01_unitedstates_128kb.mp3"
assert audio_path.exists()

In [11]:
# Create test audio
def create_test_audio():
    """Create a simple test audio signal."""
    sample_rate = 16000
    duration = 2  # seconds
    t = np.linspace(0, duration, sample_rate * duration)
    
    # Create a simple tone
    frequency = 440  # A4 note
    audio = 0.3 * np.sin(2 * np.pi * frequency * t)
    
    # Add some variation
    audio += 0.1 * np.sin(2 * np.pi * frequency * 2 * t)
    audio += 0.05 * np.random.randn(len(t))  # Add noise
    
    return AudioData(
        samples=audio.astype(np.float32),
        sample_rate=sample_rate,
        duration=len(audio) / sample_rate,
        filepath=None,
        metadata={"description": "Test tone signal"}
    )

test_audio = create_test_audio()
print(f"Created test audio: {test_audio.duration:.2f} seconds at {test_audio.sample_rate} Hz")

Created test audio: 2.00 seconds at 16000 Hz


In [12]:
if success:
    # Test with synthetic audio (may not produce meaningful text)
    try:
        print("Testing with synthetic audio...")
        result = manager.execute_plugin("gemini", test_audio)
        print("Transcription result:")
        print(f"  Text: '{result.text[:200]}...'" if len(result.text) > 200 else f"  Text: '{result.text}'")
        print(f"  Metadata: {result.metadata}")
    except Exception as e:
        print(f"Note: Synthetic audio may not produce meaningful results")
        print(f"Error: {e}")

Testing with synthetic audio...


Downsampling:   0%|          | 0.0/2.0s [00:00<?]

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Downsampled audio to 16000Hz
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcribing with Gemini model: gemini-2.5-flash (max_tokens: 65536)
google_genai.models - INFO - AFC is enabled with max remote calls: 10.



Successfully downsampled audio.


httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
google_genai.models - INFO - AFC remote call 1 is done.
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcription completed: 4 words


Transcription result:
  Text: '[Continuous high-pitched whirring sound]'
  Metadata: {'model': 'gemini-2.5-flash', 'temperature': 0.3, 'top_p': 0.95, 'max_output_tokens': 65536, 'prompt': 'Transcribe this audio accurately, including any technical terms.', 'use_file_upload': False, 'use_streaming': False}


In [13]:
if success:
    # Test with actual audio file if available
    if audio_path.exists():
        print(f"Transcribing: {audio_path}")
        
        # You can override config at execution time
        result = manager.execute_plugin(
            "gemini", 
            str(audio_path),
            temperature=0.0,  # Override for deterministic output
            prompt="Provide a detailed transcription of this audio."
        )
        
        print(f"\nTranscription (first 500 chars):")
        print(result.text[:500])
        print(f"\n...({len(result.text)} total characters)")
        print(f"\nModel used: {result.metadata.get('model')}")
    else:
        print(f"Sample audio file not found: {audio_path}")
        print("Place an audio file at this path to test real transcription.")

Transcribing: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio.mp3


Downsampling:   0%|          | 0.0/28.0s [00:00<?]

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Downsampled audio to 16000Hz
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcribing with Gemini model: gemini-2.5-flash (max_tokens: 65536)
google_genai.models - INFO - AFC is enabled with max remote calls: 10.



Successfully downsampled audio.


httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
google_genai.models - INFO - AFC remote call 1 is done.
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcription completed: 43 words



Transcription (first 500 chars):
November the 10th, Wednesday, 9 p.m.

I'm standing in a dark alley.

After waiting several hours, the time has come.

A woman with long dark hair approaches.
I have to act, and fast, before she realizes what has happened.
I must find out.

...(238 total characters)

Model used: gemini-2.5-flash


## Test Downsampling Feature

In [14]:
if success:
    # Test with downsampling enabled
    print("Testing downsampling feature...")
    
    # Update config to enable downsampling
    downsample_config = {
        "downsample_audio": True,
        "downsample_rate": 8000,  # Very low sample rate
        "downsample_channels": 1  # Mono
    }
    
    manager.update_plugin_config("gemini", downsample_config, merge=True)
    
    # Create higher quality audio
    hq_audio = AudioData(
        samples=np.random.randn(44100 * 2).astype(np.float32) * 0.1,  # 2 seconds at 44.1kHz
        sample_rate=44100,
        duration=2.0,
        filepath=None,
        metadata={"description": "High quality test audio"}
    )
    
    print(f"Original audio: {hq_audio.sample_rate} Hz")
    print(f"Will downsample to: {downsample_config['downsample_rate']} Hz")
    
    try:
        result = manager.execute_plugin("gemini", hq_audio)
        print("Downsampling and transcription successful!")
    except Exception as e:
        print(f"Note: Downsampling requires ffmpeg to be installed")
        print(f"Error: {e}")

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Cleanup completed
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"


Testing downsampling feature...


httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: gemini


Original audio: 44100 Hz
Will downsample to: 8000 Hz


Downsampling:   0%|          | 0.0/2.0s [00:00<?]

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Downsampled audio to 8000Hz
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcribing with Gemini model: gemini-2.5-flash (max_tokens: 65536)
google_genai.models - INFO - AFC is enabled with max remote calls: 10.



Successfully downsampled audio.


httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
google_genai.models - INFO - AFC remote call 1 is done.
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcription completed: 5 words


Downsampling and transcription successful!


## Test Entry Point Discovery

In [15]:
# This will work after installing the package
print("Testing entry point discovery:")

# Create a new manager for entry point testing
manager2 = PluginManager(plugin_interface=TranscriptionPlugin)

# Discover plugins via entry points
discovered = manager2.discover_plugins()
print(f"\nDiscovered {len(discovered)} plugin(s) via entry points:")
for plugin_meta in discovered:
    print(f"  - {plugin_meta.name} v{plugin_meta.version} from {plugin_meta.package_name}")

# Load discovered Gemini plugin
for plugin_meta in discovered:
    if plugin_meta.name == "gemini":
        if os.environ.get("GEMINI_API_KEY"):
            success = manager2.load_plugin(plugin_meta, config={"model": "gemini-2.5-flash"})
            print(f"\nLoaded {plugin_meta.name}: {success}")
        else:
            print(f"\nSkipping {plugin_meta.name} - API key not set")

cjm_plugin_system.core.manager.PluginManager - INFO - Discovered plugin: gemini v0.0.5 from package cjm-transcription-plugin-gemini


Testing entry point discovery:

Discovered 1 plugin(s) via entry points:
  - gemini v0.0.5 from cjm-transcription-plugin-gemini


httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Loaded plugin: gemini



Loaded gemini: True


## Test File Upload Feature

Test uploading audio files to Gemini API instead of embedding them in the request.

In [16]:
if success and audio_path.exists():
    # Test with file upload enabled
    print("Testing file upload feature...")
    
    # Update config to use file upload
    upload_config = {
        "use_file_upload": True,
        "delete_uploaded_files": True  # Automatically delete after transcription
    }
    
    manager.update_plugin_config("gemini", upload_config, merge=True)
    
    try:
        print(f"Uploading and transcribing: {audio_path}")
        result = manager.execute_plugin(
            "gemini", 
            str(audio_path),
            temperature=0.0,
            prompt="Provide a detailed transcription of this audio."
        )
        
        print(f"\nTranscription (first 300 chars):")
        print(result.text[:300])
        print(f"\nMetadata:")
        print(f"  Used file upload: {result.metadata.get('use_file_upload')}")
        print(f"  Used streaming: {result.metadata.get('use_streaming')}")
        print(f"  Model: {result.metadata.get('model')}")
        
    except Exception as e:
        print(f"Error during file upload test: {e}")

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Cleanup completed
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"


Testing file upload feature...


httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: gemini


Uploading and transcribing: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio.mp3


Downsampling:   0%|          | 0.0/28.0s [00:00<?]

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Downsampled audio to 8000Hz
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Uploading audio file: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio_downsampled.mp3



Successfully downsampled audio.


httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files?upload_id=AHVrFxM7zomZ15GJS6voBpsVqid5swQc6sYcvVlaZZTY2ff9yLewe1IjQqqKL0b740kTVPOXau5JAwEDNHfGdyhEY-LQhVVv9hZCkX3C7Mgg6MM&upload_protocol=resumable "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Successfully uploaded file: files/cbukq9f4l2wy
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcribing with Gemini model: gemini-2.5-flash (max_tokens: 65536)
google_genai.models - INFO - AFC is enabled with max remote calls: 10.
httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
google_genai.models - INFO - AFC remote call 1 is done.
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcription completed: 43 words
httpx - INFO - HTTP Req


Transcription (first 300 chars):
November the 10th, Wednesday. 9 PM.

I'm standing in a dark alley.

After waiting several hours, the time has come.

A woman with long, dark hair approaches. I have to act, and fast, before she realizes what has happened. I must find out.

Metadata:
  Used file upload: True
  Used streaming: False
  Model: gemini-2.5-flash


## Test Streaming Feature

Test streaming transcription responses for real-time output.

In [17]:
if success and audio_path.exists():
    # Test with streaming enabled
    print("Testing streaming feature...")
    
    # Update config to use streaming
    streaming_config = {
        "use_file_upload": False,  # Can combine with file upload if desired
        "use_streaming": True
    }
    
    manager.update_plugin_config("gemini", streaming_config, merge=True)
    
    try:
        print(f"Streaming transcription for: {audio_path}")
        result = manager.execute_plugin(
            "gemini", 
            str(audio_path),
            temperature=0.0,
            prompt="Transcribe this audio."
        )
        
        print(f"\nTranscription (first 300 chars):")
        print(result.text[:300])
        print(f"\nMetadata:")
        print(f"  Used file upload: {result.metadata.get('use_file_upload')}")
        print(f"  Used streaming: {result.metadata.get('use_streaming')}")
        print(f"  Total words: {len(result.text.split())}")
        
    except Exception as e:
        print(f"Error during streaming test: {e}")

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Cleanup completed
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"


Testing streaming feature...


httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: gemini


Streaming transcription for: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio.mp3


Downsampling:   0%|          | 0.0/28.0s [00:00<?]

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Downsampled audio to 8000Hz
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcribing with Gemini model: gemini-2.5-flash (max_tokens: 65536)
google_genai.models - INFO - AFC is enabled with max remote calls: 10.
google_genai.models - INFO - AFC remote call 1 is done.



Successfully downsampled audio.


httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:streamGenerateContent?alt=sse "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Streaming transcription completed
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcription completed: 43 words



Transcription (first 300 chars):
November the 10th, Wednesday, 9:00 p.m. I'm standing in a dark alley. After waiting several hours, the time has come. A woman with long, dark hair approaches. I have to act, and fast, before she realizes what has happened. I must find out.

Metadata:
  Used file upload: False
  Used streaming: True
  Total words: 43


## Test Combined Features

Test using both file upload and streaming together.

In [18]:
if success and audio_path.exists():
    # Test with both file upload and streaming
    print("Testing combined file upload + streaming...")
    
    # Update config to use both features
    combined_config = {
        "use_file_upload": True,
        "use_streaming": True,
        "delete_uploaded_files": True
    }
    
    manager.update_plugin_config("gemini", combined_config, merge=True)
    
    try:
        print(f"Uploading and streaming transcription for: {audio_path}")
        
        # You can also test with different models
        result = manager.execute_plugin(
            "gemini", 
            str(audio_path),
            model="gemini-2.5-flash",  # Or any available model
            temperature=0.0,
            prompt="Generate a detailed and accurate transcription."
        )
        
        print(f"\nTranscription (first 400 chars):")
        print(result.text[:400])
        print(f"\nMetadata:")
        for key, value in result.metadata.items():
            print(f"  {key}: {value}")
        
    except Exception as e:
        print(f"Error during combined features test: {e}")

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Cleanup completed
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"


Testing combined file upload + streaming...


httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: gemini


Uploading and streaming transcription for: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio.mp3


Downsampling:   0%|          | 0.0/28.0s [00:00<?]

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Downsampled audio to 8000Hz
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Uploading audio file: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio_downsampled.mp3



Successfully downsampled audio.


httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files?upload_id=AHVrFxNpqktS1Eg5ElxmHZwtzy-tE6IC-5RZfHiXoRL-WTRLWTmG7LDA9aTDTaT8CA-nT5SQj9FM7YR5HfYCs1cbK78BHcfEHVzRcnTX5IYRwA&upload_protocol=resumable "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Successfully uploaded file: files/r72xq4isnock
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcribing with Gemini model: gemini-2.5-flash (max_tokens: 65536)
google_genai.models - INFO - AFC is enabled with max remote calls: 10.
google_genai.models - INFO - AFC remote call 1 is done.
httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:streamGenerateContent?alt=sse "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Streaming transcription completed
cjm_transc


Transcription (first 400 chars):
November the 10th, Wednesday, 9 pm.

I'm standing in a dark alley.

After waiting several hours, the time has come.

A woman with long, dark hair approaches.

I have to act, and fast, before she realizes what has happened. I must find out.

Metadata:
  model: gemini-2.5-flash
  temperature: 0.0
  top_p: 0.95
  max_output_tokens: 65536
  prompt: Generate a detailed and accurate transcription.
  use_file_upload: True
  use_streaming: True


## Test File Management

Test that uploaded files are properly tracked and cleaned up.

In [19]:
if success:
    # Test file management with multiple uploads
    print("Testing file management and cleanup...")
    
    # Create a test plugin instance directly to track uploaded files
    test_plugin = manager.get_plugin("gemini")
    
    # Configure to NOT auto-delete to test manual cleanup
    no_delete_config = {
        "use_file_upload": True,
        "use_streaming": False,
        "delete_uploaded_files": False  # Keep files for manual deletion
    }
    
    manager.update_plugin_config("gemini", no_delete_config, merge=True)
    
    try:
        # Upload and transcribe without auto-deletion
        print("Uploading file without auto-deletion...")
        result = manager.execute_plugin(
            "gemini", 
            str(audio_path),
            prompt="Quick transcription test."
        )
        
        # Check tracked files
        print(f"\nTracked uploaded files: {len(test_plugin.uploaded_files)}")
        if test_plugin.uploaded_files:
            for file in test_plugin.uploaded_files:
                print(f"  - {file.name}")
        
        # Now test cleanup
        print("\nManually cleaning up...")
        test_plugin.cleanup()
        
        print(f"Files after cleanup: {len(test_plugin.uploaded_files)}")
        
    except Exception as e:
        print(f"Error during file management test: {e}")
    
    # Reset to auto-delete
    manager.update_plugin_config("gemini", {"delete_uploaded_files": True}, merge=True)

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Cleanup completed
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"


Testing file management and cleanup...


httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: gemini


Uploading file without auto-deletion...


Downsampling:   0%|          | 0.0/28.0s [00:00<?]

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Downsampled audio to 8000Hz
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Uploading audio file: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio_downsampled.mp3



Successfully downsampled audio.


httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files?upload_id=AHVrFxMf0Ttzu541qZLIy884eHbogq5l-d_9gh5fb6ktsUi7ZfYosnenFManJd1jn1s3Yd9dkvTa42sL5ssDxxDtVjBfNOc7U8o7nTcJjJGJDw&upload_protocol=resumable "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Successfully uploaded file: files/w8osqm5qlpln
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcribing with Gemini model: gemini-2.5-flash (max_tokens: 65536)
google_genai.models - INFO - AFC is enabled with max remote calls: 10.
httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
google_genai.models - INFO - AFC remote call 1 is done.
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Transcription completed: 43 words
cjm_transcription_plugin


Tracked uploaded files: 1
  - files/w8osqm5qlpln

Manually cleaning up...
Files after cleanup: 0


httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models?pageToken=Cittb2RlbHMvZ2VtaW5pLTIuNS1mbGFzaC1uYXRpdmUtYXVkaW8tbGF0ZXN0 "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Found 26 audio-capable models
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Updated max_output_tokens to 65536 for model 'gemini-2.5-flash'
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Initialized Gemini plugin with model 'gemini-2.5-flash'
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: gemini


## Test Streaming with execute_stream Method

Test the new execute_stream method that yields chunks in real-time.

In [20]:
if success and audio_path.exists():
    # Test the new execute_stream method
    print("Testing execute_stream method...")
    
    # Check if plugin supports streaming
    supports_streaming = manager.check_streaming_support("gemini")
    print(f"Plugin supports streaming: {supports_streaming}")
    
    if supports_streaming:
        print(f"\nStreaming transcription for: {audio_path}")
        print("Chunks as they arrive:")
        print("-" * 40)
        
        chunks = []
        chunk_count = 0
        
        # Stream transcription chunks
        for chunk in manager.execute_plugin_stream("gemini", str(audio_path)):
            chunk_count += 1
            chunks.append(chunk)
            print(f"[Chunk {chunk_count}]: {chunk[:50]}..." if len(chunk) > 50 else f"[Chunk {chunk_count}]: {chunk}")
        
        print("-" * 40)
        print(f"\nTotal chunks received: {chunk_count}")
        print(f"Complete transcription: {''.join(chunks)[:200]}...")
        
        # Get list of all streaming-capable plugins
        streaming_plugins = manager.get_streaming_plugins()
        print(f"\nAll plugins with streaming support: {streaming_plugins}")

cjm_plugin_system.core.manager.PluginManager - INFO - Using streaming mode for plugin gemini


Testing execute_stream method...
Plugin supports streaming: True

Streaming transcription for: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio.mp3
Chunks as they arrive:
----------------------------------------


Downsampling:   0%|          | 0.0/28.0s [00:00<?]

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Downsampled audio to 8000Hz
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Uploading audio file: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-gemini/test_files/short_test_audio_downsampled.mp3



Successfully downsampled audio.


httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/upload/v1beta/files?upload_id=AHVrFxMYtogTmGselVcVlgcXtlbzdHM-nkg4MmwODViKanfDF76S3977FwkCfFNcJ9oKPRiUmt2dHzMix3Mb-JYYk1fyPln61z02-xYOgpZPxw&upload_protocol=resumable "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Successfully uploaded file: files/44h447b04u7z
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Streaming transcription with Gemini model: gemini-2.5-flash (max_tokens: 65536)
google_genai.models - INFO - AFC is enabled with max remote calls: 10.
google_genai.models - INFO - AFC remote call 1 is done.
httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:streamGenerateContent?alt=sse "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Streaming completed: 2 chunks, 43

[Chunk 1]: November the 10th, Wednesday, 9:00 p.m.

I'm stand...
[Chunk 2]:  have to act, and fast, before she realizes what h...


httpx - INFO - HTTP Request: DELETE https://generativelanguage.googleapis.com/v1beta/files/44h447b04u7z "HTTP/1.1 200 OK"
cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Deleted uploaded file: files/44h447b04u7z


----------------------------------------

Total chunks received: 2
Complete transcription: November the 10th, Wednesday, 9:00 p.m.

I'm standing in a dark alley.

After waiting several hours, the time has come.

A woman with long dark hair approaches. I have to act, and fast, before she rea...

All plugins with streaming support: ['gemini']


## Cleanup

In [21]:
if success:
    # Clean up
    print("Cleaning up...")
    manager.unload_plugin("gemini")
    print(f"Plugins remaining: {len(manager.list_plugins())}")

cjm_transcription_plugin_gemini.plugin.GeminiPlugin - INFO - Cleanup completed
cjm_plugin_system.core.manager.PluginManager - INFO - Unloaded plugin: gemini


Cleaning up...
Plugins remaining: 0
